### Instalando pacotes

In [21]:

!pip install openpyxl
!pip install pandas
!pip install numpy

In [22]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [23]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")


### Estrutura da Busca Local

In [24]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break
    return solucao_atual, custo_atual


### Heurísticas

In [25]:
def shift(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            vert_shift = solucao_cp.pop(idx_vert_shift)
            solucao_cp.insert(idx_destino_shift, vert_shift)
            yield solucao_cp

def swap(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            vert_swap = solucao_cp[idx_vert_swap]
            solucao_cp[idx_vert_swap] = solucao_cp[idx_destino_swap]
            solucao_cp[idx_destino_swap] = vert_swap
            # print("Solucao do swap : ", solucao_cp)
            yield solucao_cp

def inversao(solucao):
    solucao_cp = solucao[:]
    for idx_vert_inicio in range(1, len(solucao) - 1):
        for idx_destino in range(2, len(solucao) - 1):
            sublista_invertida = solucao_cp[idx_vert_inicio:idx_destino]
            sublista_invertida.reverse()
            yield solucao_cp[0:idx_vert_inicio] + sublista_invertida + solucao_cp[idx_destino:]


## Busca por vizinho mais próximo

In [26]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total


## Aplicando nos problemas

In [27]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out = None):
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_custo = custo
    melhor_caminho = caminho
    heuristica_melhor_solucao = None
    out = True if out is None else out
    if out:
        print("Aplicando Heuristica Construtiva: ", heuristica_construtiva.__name__)
        print(f"Caminho: {caminho}")
        print(f"Custo: {custo}")
        print("Aplicando busca(s) local(is):")
    for h in heuristicas_busca_local:
        solucao_local, custo_local = busca_local_primeira_melhoria(melhor_caminho, melhor_custo, h, grafo)
        if out:
            print(f"Heuristica : {h.__name__}")
            print("Tipo : Primeira Melhoria")
            print(f"Melhor caminho pos busca local: {solucao_local}")
            print(f"Melhor custo pos busca local: {custo_local}")
        if custo_local < melhor_custo:
            melhor_caminho = solucao_local
            melhor_custo = custo_local
            heuristica_melhor_solucao = h.__name__
    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [28]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2291.3999999999996
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos bus


### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.


In [29]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo: 2355.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 2, 3, 4, 41, 30, 6

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [30]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo: 1951.4999999999998
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [31]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo: 1950.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 17, 20, 33, 19, 34, 16, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 30, 6, 27, 1]
Melhor custo pos busca local: 1

### Insercao Mais Barata

In [32]:
import math

# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = grafo[vertice_inicial].idxmin()

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin()

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)



### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [33]:
melhor_custo, melhor_caminho, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo: 2148.1
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2148.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 21, 8, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2147.2
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 2, 11, 5, 46, 14, 43, 

## Algoritmo Genetico

### Gerador de solucoes aleatorias

In [34]:
import random
def solucao_aleatoria(grafo : pd.DataFrame, vertice_inicial):
    rota = list(map(lambda i : i + 1,grafo.index))
    rota.remove(vertice_inicial)
    random.shuffle(rota)
    return [vertice_inicial] + rota + [vertice_inicial]

# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        # print("v1: ", v1)
        # print("v2: ", v2)
        custo += grafo[v1][v2 - 1]
    return custo

samples = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,16)]
print(samples)
custos = [calcular_custo(solucao, matriz_problema_km) for solucao in samples]
print(f"Minimo : {min(custos)}")
print(f"Maximo : {max(custos)}")


[[1, 13, 35, 33, 42, 38, 22, 41, 24, 9, 27, 47, 18, 32, 30, 23, 7, 3, 40, 4, 16, 17, 36, 12, 48, 31, 43, 2, 45, 8, 26, 15, 28, 14, 46, 20, 11, 37, 29, 34, 5, 19, 10, 39, 44, 25, 6, 21, 1], [1, 5, 12, 43, 36, 13, 21, 37, 44, 48, 8, 30, 47, 27, 16, 46, 28, 41, 24, 42, 35, 45, 11, 20, 9, 7, 10, 34, 22, 32, 17, 18, 25, 23, 26, 6, 38, 40, 3, 14, 15, 4, 2, 33, 39, 29, 19, 31, 1], [1, 6, 3, 13, 19, 23, 14, 45, 24, 28, 12, 36, 44, 32, 42, 9, 48, 33, 38, 25, 4, 16, 21, 39, 8, 5, 47, 43, 20, 7, 35, 30, 34, 40, 10, 37, 29, 26, 46, 41, 18, 15, 27, 31, 22, 11, 2, 17, 1], [1, 48, 29, 3, 33, 16, 6, 27, 10, 18, 8, 17, 19, 2, 32, 7, 31, 4, 23, 40, 47, 38, 14, 15, 11, 24, 26, 20, 9, 41, 36, 39, 44, 45, 22, 25, 34, 28, 37, 35, 42, 46, 30, 21, 12, 43, 5, 13, 1], [1, 45, 36, 48, 31, 22, 30, 15, 19, 43, 10, 33, 42, 40, 5, 13, 9, 35, 32, 44, 3, 37, 14, 8, 47, 6, 18, 26, 12, 20, 38, 46, 24, 21, 2, 11, 17, 7, 29, 41, 28, 39, 34, 25, 27, 23, 4, 16, 1], [1, 44, 35, 39, 9, 12, 31, 11, 27, 47, 17, 14, 45, 38, 18, 

## Gerando populacao
Gerar 15 indivíduos

### Populacao para o problema 1

In [35]:
solucao_insercao_mais_barata = insercao_mais_barata(matriz_problema_km, 1)[0]
solucao_vizinho_mais_proximo = vizinho_mais_proximo(matriz_problema_km, 1)[0]
solucao_insercao_mais_barata_local = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao], out=False)[0]
solucao_vizinho_mais_proximo_local = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao], out=False)[0]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,16)]

samples = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

print(samples)
# Ordenando pra facilitar a selecao por elitismo
samples = sorted(samples, key=lambda solucao: calcular_custo(solucao, matriz_problema_km))


[[1, 18, 13, 20, 34, 28, 43, 41, 27, 5, 39, 25, 2, 30, 7, 46, 36, 14, 32, 38, 40, 33, 22, 9, 44, 11, 10, 31, 3, 16, 24, 6, 29, 48, 45, 8, 37, 12, 42, 35, 15, 23, 21, 47, 19, 17, 26, 4, 1], [1, 38, 35, 4, 19, 14, 7, 12, 43, 18, 31, 40, 47, 6, 15, 24, 3, 23, 9, 28, 48, 25, 21, 36, 11, 2, 41, 16, 29, 37, 30, 45, 17, 20, 10, 34, 44, 22, 5, 33, 13, 42, 27, 26, 39, 8, 46, 32, 1], [1, 45, 24, 43, 17, 16, 12, 44, 34, 31, 26, 30, 48, 3, 36, 39, 25, 7, 22, 2, 19, 9, 5, 13, 23, 33, 41, 42, 6, 18, 38, 15, 11, 14, 28, 47, 35, 20, 29, 8, 27, 46, 21, 37, 40, 32, 10, 4, 1], [1, 31, 37, 38, 48, 30, 26, 12, 28, 18, 46, 2, 36, 45, 33, 42, 44, 35, 23, 43, 3, 24, 17, 32, 41, 27, 5, 39, 25, 29, 11, 14, 16, 8, 6, 10, 20, 19, 9, 4, 40, 22, 34, 15, 21, 13, 7, 47, 1], [1, 29, 41, 4, 37, 5, 22, 3, 19, 26, 8, 11, 43, 45, 36, 46, 47, 16, 21, 33, 38, 39, 24, 10, 14, 35, 25, 17, 9, 20, 12, 48, 34, 40, 32, 30, 28, 23, 7, 27, 44, 31, 13, 2, 42, 6, 15, 18, 1], [1, 7, 35, 24, 32, 10, 27, 47, 33, 31, 26, 17, 14, 34, 36, 

## 1) Inicio
Organizando os cromossomos

In [36]:
cromossomos = [(solucao, calcular_custo(solucao, matriz_problema_km)) for solucao in samples]


## Metodos de Selecao

In [37]:
def elitismo(populacao : list, num_pares):
    populacao_ordenada = sorted(populacao, key=lambda x: x[1])
    genitores = []
    for i in range(0,num_pares):
        genitores.append((populacao_ordenada[i], populacao_ordenada[i + 1]))
    return genitores

# def torneio(populacao : list, num_ num_candidatos = None):
#     for
#     candidatos = random.sample(populacao[1:-1], num_candidatos)
    # populacao_ordenada = sorted(populacao, key=lambda x: x[1])
    # genitores = []
    # for i in range(0,num_pares):
    #     genitores.append((populacao_ordenada[i], populacao_ordenada[i + 1]))
    # return genitores
def torneio(populacao: list, num_pares: int, tamanho_torneio: int = 3):
  genitores = []
  for _ in range(num_pares):
    competidores_1 = random.sample(populacao, k=tamanho_torneio)
    pai1 = min(competidores_1, key=lambda x: x[1])
    competidores_2 = random.sample(populacao, k=tamanho_torneio)
    pai2 = min(competidores_2, key=lambda x: x[1])
    genitores.append((pai1, pai2))
  return genitores



### Metodos de Cruzamento

In [38]:
def crossover_1_ponto(cromossomo_1, cromossomo_2):
    n = len(cromossomo_1) # Cromossomos tem comprimentos iguais
    # Ponto de corte
    mid = n // 2
    offspring_1 = cromossomo_1[0:mid] + cromossomo_2[mid:n]
    offspring_2 = cromossomo_2[0:mid] + cromossomo_1[mid:n]
    offsprings = [offspring_1, offspring_2]
    # As labels sao numeros em sequencia
    vertices = set(range(2, n))
    for i, offspring in enumerate(offsprings):
        # vertices ja presentes no offspring atual
        vertices_no_offspring = set(offspring)
        vertices_no_offspring.remove(1)
        # se o numero de elementos unicos eh menor que o tamanho - 1 (ja que eh um ciclo),
        # ja elementos repetidos
        if len(vertices_no_offspring) < n-1:
            vertices_restantes = vertices.difference(vertices_no_offspring)
            novo_offspring = offspring[0:mid]
            controle = set(offspring[1:mid])
            for v in offspring[mid:]:
                novo_v = None
                if v in controle:
                    novo_v = vertices_no_offspring.pop()
                else:
                    novo_v = v
                controle.add(novo_v)
                novo_offspring.append(novo_v)
            offsprings[i] = novo_offspring
    return offsprings[0], offsprings[1]

def crossover_ox(genitor_1, genitor_2):
    offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    return offspring_1, offspring_2



def cruzamento(genitores, grafo):
    offsprings = []
    for g in genitores:
        genitor_1,genitor_2=g
        offspring1, offspring2 = crossover_ox(genitor_1[0], genitor_2[0])
        offsprings.extend([(offspring1, calcular_custo(offspring1, grafo)), (offspring2,calcular_custo(offspring1, grafo))])
    return offsprings

In [39]:
import random
# def mutacao_swap(cromossomos, probabilidade, grafo):
#     novos_cromossomos = []
#     for cromossomo in cromossomos:
#         while random.random() < probabilidade:
#             print("Vai haver mutacao")
#             while True: # Loop para tratar a mesma ser escolhida swap
#                 gene_1 = random.choice(range(1, len(cromossomo[0]) - 1))
#                 gene_2 = random.choice(range(1, len(cromossomo[0]) - 1))
#                 if gene_1 != gene_2:
#                     temp = cromossomo[0][gene_1]
#                     cromossomo[0][gene_1] = cromossomo[0][gene_2]
#                     cromossomo[0][gene_2] = temp
#                     novos_cromossomos.append((cromossomo[0], calcular_custo(cromossomo[0], grafo)))
#                     break
#         # Nao faz mutacao
#         novos_cromossomos.append(cromossomo)
#         continue
#     return novos_cromossomos
def mutacao_swap(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo in cromossomos:
        cromossomo = list(cromossomo[0]) # extrai o caminho

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            cromossomo[gene_1], cromossomo[gene_2] = cromossomo[gene_2], cromossomo[gene_1]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos

def mutacao_inversao(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo_original_tuple in cromossomos:
        cromossomo = list(cromossomo_original_tuple[0]) # Trabalhar com a lista da rota

        if random.random() < probabilidade:
            # Aplica a mutação (swap)
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            gene_inicio = min(gene_1, gene_2)
            gene_fim = max(gene_1, gene_2)
            cromossomo = cromossomo[:gene_inicio] + cromossomo[gene_inicio:gene_fim] + cromossomo[gene_fim:]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos




In [40]:
# for custo, cromossomo in cromossomos.items():
#     mutacao_swap(cromossomo, 0.5)

In [41]:
a = {1:"a"}
b = random.choice(list(a.items()))
b

(1, 'a')

In [42]:
def renovacao(populacao_atual, offspring, num_populacao):
    nova_populacao = list()
#     offspring_ordenada = sorted(offspring, key=lambda a : a[1])
#     pop_ordenada = sorted(populacao_atual, key=lambda a : a[1])
#     pop_ordenada[len(pop_ordenada) // 2:] = offspring_ordenada[:len(pop_ordenada) // 2]
#     return pop_ordenada
    # Usando torneio
    while len(nova_populacao) < num_populacao:
        pop_individuo = random.choice(populacao_atual)
        offspring_individuo = random.choice(list(offspring))
        escolhido = min(offspring_individuo, pop_individuo, key=lambda x: x[1])
        nova_populacao.append(escolhido)
    return nova_populacao



In [43]:

print(float(random.random()))

0.06515146651487824


def algoritmo_genetico(numero_iteracoes, taxa_mutacao, mutacoes, selecao,

In [44]:
def algoritmo_genetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao):
    populacao_atual = populacao_inicial
    for iter in range(0, numero_iteracoes):
        genitores = selecao(populacao_atual, len(populacao_atual) // 2)
        print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        # Ja aplica a taxa de mutacao a populacao
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        print(f"Tamanho do offspring: {len(offsprings)}")
        print(offsprings)
        populacao_atual = renovacao(populacao_atual, offsprings, len(populacao_atual))
        print(f"Tamanho da pop: {len(populacao_atual)}")
    return populacao_atual

resultado = algoritmo_genetico(matriz_problema_km, cromossomos, 20, 0.3, mutacao_inversao, elitismo, cruzamento, renovacao)
solucao = min(resultado, key=lambda x : x[1])
print(solucao)

Numero de genitores: 10
Tamanho do offspring: 20
[([1, 2, 11, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 20, 33, 39, 35, 42, 26, 32, 16, 34, 19, 7, 23, 4, 3, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 17, 41, 30, 6, 27, 44, 28, 15, 18, 45], np.float64(2310.9)), ([1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 7, 19, 17, 20, 33, 16, 34, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 39, 35, 42, 26, 32, 30, 6, 27, 28, 15, 18, 45, 44, 41], np.float64(2427.8)), ([1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 7, 19, 17, 20, 33, 16, 34, 42, 26, 32, 35, 39, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 30, 6, 27, 28, 15, 18, 45, 44, 41], np.float64(2069.2999999999997)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 23, 4, 3, 2, 20, 47, 37, 38, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45], np.float64(2370.7999999999997)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 2, 3

## Memetico

In [46]:
def algoritmo_memetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao, heuristicas_busca_local):
    populacao_atual = populacao_inicial

    for iter in range(numero_iteracoes):
        print(f"\n--- Geração {iter + 1} ---")
        genitores = selecao(populacao_atual, len(populacao_atual) // 2)
        print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        print(f"Tamanho do offspring: {len(offsprings)}")
        print(offsprings)

        novos_offsprings = []

        for i, item in enumerate(offsprings):
            if isinstance(item, tuple):
                individuo = item[0]
            else:
                individuo = item

            custo_individuo = calcular_custo(individuo, grafo)

            # Dica: printar aqui ajuda a ver que não travou
            print(f"\r  > Otimizando indivíduo {i+1}/{len(offsprings)}...", end="")

            for heuristica in heuristicas_busca_local:
                nova_rota, novo_custo = busca_local_primeira_melhoria(individuo, custo_individuo, heuristica, grafo)
                individuo = nova_rota
                custo_individuo = novo_custo

            novos_offsprings.append((individuo, custo_individuo))

        print(f"Tamanho da pop final: {len(populacao_atual)}")

        # Opcional: Mostrar o melhor da geração atual
        melhor_da_geracao = min(populacao_atual, key=lambda x: x[1])
        print(f"Melhor desta geração: {melhor_da_geracao[1]:.2f}")

    return populacao_atual

In [ ]:
resultado = algoritmo_memetico(
    grafo=matriz_problema_km,
    populacao_inicial=cromossomos,
    numero_iteracoes=50,
    taxa_mutacao=0.3,
    mutacao=mutacao_inversao,
    selecao=elitismo,
    cruzamento=cruzamento,
    renovacao=renovacao,
    heuristicas_busca_local=[swap, shift, inversao]
)

print(resultado)


--- Geração 1 ---
Numero de genitores: 10
Tamanho do offspring: 20
[([1, 2, 11, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 20, 33, 39, 35, 42, 26, 32, 16, 34, 19, 7, 23, 4, 3, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 17, 41, 30, 6, 27, 44, 28, 15, 18, 45], np.float64(2310.9)), ([1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 7, 19, 17, 20, 33, 16, 34, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 39, 35, 42, 26, 32, 30, 6, 27, 28, 15, 18, 45, 44, 41], np.float64(2427.8)), ([1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 7, 19, 17, 20, 33, 16, 34, 42, 26, 32, 35, 39, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 30, 6, 27, 28, 15, 18, 45, 44, 41], np.float64(2069.2999999999997)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 23, 4, 3, 2, 20, 47, 37, 38, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45], np.float64(2370.7999999999997)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 3